 # Stroke Rehab EEG Analysis Pipeline



 Pipeline to convert .mat files into MNE Raw and Epochs objects and store them in a structured DataFrame.

 ## 🧰 Setups and Imports

In [4]:
import os
import re
import mne
import numpy as np
import pandas as pd
from scipy.io import loadmat


 ## ⚙️ Constants Definition

In [2]:
DATA_DIR = './stroke-rehab-data-analysis/data/stroke-rehab'
FILE_REGEX = r'(?P<subject>P\d+)_(?P<stage>pre|post)_(?P<split>training|test)\.mat'
CHANNEL_NAMES = ['FC3','FCz','FC4','C5','C3','C1','Cz','C2','C4','C6', 'CP3','CP1','CPz','CP2','CP4','Pz']
CHANNEL_TYPES = ['eeg'] * len(CHANNEL_NAMES)
MONTAGE = 'standard_1020'
EVENT_ID={'left': 1, 'right': 2}


 ## 📂 Data File Paths Parsing

In [5]:
file_entries = []

for fname in os.listdir(DATA_DIR):
    match = re.match(FILE_REGEX, fname)
    if match:
        file_entries.append({
            'filepath': os.path.join(DATA_DIR, fname),
            'subject': match.group('subject'),
            'stage': match.group('stage'),
            'split': match.group('split'),
        })

df = pd.DataFrame(file_entries)
df.head(10)


,filepath,subject,stage,split
0,./stroke-rehab-data-analysis/data/stroke-rehab...,P1,post,test
1,./stroke-rehab-data-analysis/data/stroke-rehab...,P1,post,training
2,./stroke-rehab-data-analysis/data/stroke-rehab...,P1,pre,test
3,./stroke-rehab-data-analysis/data/stroke-rehab...,P1,pre,training
4,./stroke-rehab-data-analysis/data/stroke-rehab...,P2,post,test
5,./stroke-rehab-data-analysis/data/stroke-rehab...,P2,post,training
6,./stroke-rehab-data-analysis/data/stroke-rehab...,P2,pre,test
7,./stroke-rehab-data-analysis/data/stroke-rehab...,P2,pre,training
8,./stroke-rehab-data-analysis/data/stroke-rehab...,P3,post,test
9,./stroke-rehab-data-analysis/data/stroke-rehab...,P3,post,training


 ## 🧠 MNE Raw Objects Generation

In [6]:
def make_info(subject, stage, split, fs):
    """Create MNE info object with metadata."""
    info = mne.create_info(
        ch_names=CHANNEL_NAMES,
        sfreq=fs,
        ch_types=CHANNEL_TYPES
    )
    info.set_montage(MONTAGE)

    # Add metadata
    info['subject_info'] = {'his_id': subject}
    info['description'] = str({'stage': stage, 'split': split})

    return info

def make_annotations(triggers, fs):
    """Create annotations for the raw data."""
    # Create annotations based on the triggers
    padded = np.r_[0, triggers, 0]
    diffs = np.diff(padded)
    idx = np.where(diffs != 0)[0]
    onsets, offsets = idx[::2], idx[1::2]
    values = triggers[onsets]

    onset_times = onsets / fs
    annot_durations = (offsets - onsets) / fs
    annot_descriptions = ['left' if val == 1 else 'right' for val in values]

    annot = mne.Annotations(onset=onset_times,
                            duration=annot_durations,
                            description=annot_descriptions)
    
    return annot

def load_raw_from_mat(filepath, subject, stage, split):
    """Load raw data from .mat file."""
    mat = loadmat(filepath)
    data = mat['y'].T
    triggers = mat['trig'].ravel()
    fs = float(mat['fs'].squeeze())
    
    info = make_info(subject, stage, split, fs)

    raw = mne.io.RawArray(data, info)

    annot = make_annotations(triggers, fs)
    
    raw.set_annotations(annot)

    return raw


In [7]:
df['raw'] = df.apply(
    lambda row: load_raw_from_mat(row['filepath'], row['subject'], row['stage'], row['split']),
    axis=1
)


Creating RawArray with float64 data, n_channels=16, n_times=194088
    Range : 0 ... 194087 =      0.000 ...   758.152 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=197343
    Range : 0 ... 197342 =      0.000 ...   770.867 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=204560
    Range : 0 ... 204559 =      0.000 ...   799.059 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=271816
    Range : 0 ... 271815 =      0.000 ...  1061.777 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=233576
    Range : 0 ... 233575 =      0.000 ...   912.402 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=216720
    Range : 0 ... 216719 =      0.000 ...   846.559 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=199552
    Range : 0 ... 199551 =      0.000 ...   779.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=223112
    Range : 0 ..

In [8]:
# Select the simple string columns
meta = df[["subject", "stage", "split"]]
# Create a new DataFrame with the types of the objects
types = df[["raw"]].map(lambda x: type(x).__name__)
# Concatenate both for display
pd.concat([meta, types], axis=1)

,subject,stage,split,raw
0,P1,post,test,RawArray
1,P1,post,training,RawArray
2,P1,pre,test,RawArray
3,P1,pre,training,RawArray
4,P2,post,test,RawArray
5,P2,post,training,RawArray
6,P2,pre,test,RawArray
7,P2,pre,training,RawArray
8,P3,post,test,RawArray
9,P3,post,training,RawArray


 ## ✂️ MNE Epochs Objects Generation

In [9]:
def create_epochs_from_raw(raw):
    fs = raw.info['sfreq']
    events, event_id = mne.events_from_annotations(raw, event_id=EVENT_ID)
    events[:, 0] += int(2 * fs)  # Shift events forward 2s per task description
    epochs = mne.Epochs(raw, events, event_id=event_id)
    return epochs


In [10]:
df['epochs'] = df['raw'].apply(create_epochs_from_raw)


Used Annotations descriptions: [np.str_('left'), np.str_('right')]
Not setting metadata
80 matching events found
Setting baseline interval to [-0.19921875, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: [np.str_('left'), np.str_('right')]
Not setting metadata
80 matching events found
Setting baseline interval to [-0.19921875, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: [np.str_('left'), np.str_('right')]
Not setting metadata
80 matching events found
Setting baseline interval to [-0.19921875, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: [np.str_('left'), np.str_('right')]
Not setting metadata
80 matching events found
Setting baseline interval to [-0.19921875, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: [np.str_('left'), np.str_('right'

 ## 🧾 Final DataFrame Structure

In [11]:
# Select the simple string columns
meta = df[["subject", "stage", "split"]]
# Create a new DataFrame with the types of the objects
types = df[["raw","epochs"]].map(lambda x: type(x).__name__)
# Concatenate both for display
pd.concat([meta, types], axis=1)

,subject,stage,split,raw,epochs
0,P1,post,test,RawArray,Epochs
1,P1,post,training,RawArray,Epochs
2,P1,pre,test,RawArray,Epochs
3,P1,pre,training,RawArray,Epochs
4,P2,post,test,RawArray,Epochs
5,P2,post,training,RawArray,Epochs
6,P2,pre,test,RawArray,Epochs
7,P2,pre,training,RawArray,Epochs
8,P3,post,test,RawArray,Epochs
9,P3,post,training,RawArray,Epochs


In [ ]:
#test
raw_list=df['raw'].tolist()
raw=raw_list[0]

In [ ]:

def filtration(raw,l_freq,h_freq):
    raw_filtered=raw.copy().filter(l_freq=0.5, h_freq=40.)
    return(raw_filtered)

In [ ]:
def apply_ica(raw):
    ica = ICA(n_components=raw.get_data().shape[0], random_state=42, max_iter='auto')
    ica.fit(raw_filt)
    return ica



Effective window size : 1.000 (s)


In [ ]:
def BLI(raw):
    psd, freqs=mne.time_frequency.psd_array_welch(raw.get_data(), sfreq=256,fmin=0.5,fmax=40.,n_fft=256)
    psds_band = psd.mean(axis=1) 
    channel_pairs = [
        ('FC3', 'FC4'),
        ('C5', 'C6'),
        ('C3', 'C4'),
        ('C1', 'C2'),
        ('CP3', 'CP4'),
        ('CP1', 'CP2')
    ]
    ch_names=raw.info['ch_names']
    bsi_vals = []
    for ch_left, ch_right in channel_pairs:
        if ch_left in ch_names and ch_right in ch_names:
            idx_left = ch_names.index(ch_left)
            idx_right = ch_names.index(ch_right)
            P_left = psds_band[idx_left]
            P_right = psds_band[idx_right]
            denom = P_left + P_right
            if denom > 0:
                bsi = abs(P_left - P_right) / denom
                bsi_vals.append(bsi)
    return(bsi_vals)
